In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

c:\Users\Ayush\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Specify the dataset name and the column containing the content
dataset_name = "Ayush2312/deduplicated_orca_post_processed"
page_content_column = "formatted_example"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

# Display the first 15 entries
data[:2]

c:\Users\Ayush\anaconda3\envs\NLP\Lib\site-packages\datasets\load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(


[Document(page_content='"### Instruction: You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.\\n\\n### Input:\\n Multi-choice question: What is the sentiment of the following tweet?\\nTweet: @nikkigreen I told you \\nChoose your answer from:\\n + negative;\\n + positive;\\n\\n### Response:\\nTo determine the sentiment of the tweet, we need to analyze it thoroughly.\\n\\nTweet: @nikkigreen I told you\\n\\nStep 1: Identify the words or phrases that carry emotional weight.\\nIn this tweet, there is only one phrase worth examining: \\"I told you.\\"\\n\\nStep 2: Determine the sentiment of the identified words or phrases.\\n\\"I told you\\" can carry a variety of sentiments, depending on the context. It could be positive, negative, or neutral.\\n\\nStep 3: Consider the overall context of the tweet.\\nUnfortunately, without more context, it is impossible to det

In [4]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [5]:
docs[0]

Document(page_content='"### Instruction: You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.\\n\\n### Input:\\n Multi-choice question: What is the sentiment of the following tweet?\\nTweet: @nikkigreen I told you \\nChoose your answer from:\\n + negative;\\n + positive;\\n\\n### Response:\\nTo determine the sentiment of the tweet, we need to analyze it thoroughly.\\n\\nTweet: @nikkigreen I told you\\n\\nStep 1: Identify the words or phrases that carry emotional weight.\\nIn this tweet, there is only one phrase worth examining: \\"I told you.\\"\\n\\nStep 2: Determine the sentiment of the identified words or phrases.\\n\\"I told you\\" can carry a variety of sentiments, depending on the context. It could be positive, negative, or neutral.\\n\\nStep 3: Consider the overall context of the tweet.\\nUnfortunately, without more context, it is impossible to dete

In [8]:
import os
os.system("huggingface-cli login --token %HF_TOKEN%")

^C


In [7]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

c:\Users\Ayush\anaconda3\envs\NLP\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
db = FAISS.from_documents(docs, embeddings)

In [9]:
question = "tell me about Visionary Voyage"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

contribute significantly to the viewer's immersion in the fantastical world depicted in the film. Every frame of the movie is a tribute to the seamless symbiosis between Wilson's creative vision and the mastery of the film's technical team.\n\nThe Visionary Voyage also carries an underlying message of embracing one's vulnerability and conquering fear. The film's universal appeal is a testament to its capacity to resonate with audiences across demographics.\n\nIn conclusion, The Visionary Voyage is a rare gem of cinema that succeeds in blending visual enchantment and inspiring storytelling. The visionary director, Samantha Wilson, alongside the phenomenal cast and crew, create a mesmerizing experience that will undoubtedly leave its viewers contemplating the significance of embracing their fears and realizing their potential for a long time to come."


In [10]:
!pip install -U langchain-together langchain-core langchain-community faiss-cpu tiktoken

   ---------------------------------------- 0.0/256.9 kB ? eta -:--:--
   ------ --------------------------------- 41.0/256.9 kB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 194.6/256.9 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 256.9/256.9 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.8 MB 4.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 6.4 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 7.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 7.0 MB/s eta 0:00:01
   -------------------- ------------------- 0.9/1.8 MB 4.7 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 5.3 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 5.3 MB/s eta 0:00:01
   -------------------------------- ------- 1.5/1.8 MB 4.6 MB/s eta 0:00:01
   ----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.81 requires packaging==23.1, but you have packaging 23.2 which is incompatible.
autotrain-advanced 0.6.81 requires pydantic==2.4.2, but you have pydantic 2.6.1 which is incompatible.
autotrain-advanced 0.6.81 requires tiktoken==0.5.1, but you have tiktoken 0.6.0 which is incompatible.
autotrain-advanced 0.6.81 requires tqdm==4.65.0, but you have tqdm 4.66.2 which is incompatible.


In [30]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_together import TogetherEmbeddings
from langchain_community.llms import Together
import os
together_api_key = os.environ.get('TOGETHER_API_KEY')
retriever = db.as_retriever()

model = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=128,
    top_k=50,
    together_api_key = together_api_key
)

# Provide a template following the LLM's original chat template.
template = """<s>[INST] Answer my question and your memory is stored in context, if you use context for answering then start you response with "i remember" or something similar:
{context}

Question: {question} [/INST]
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [32]:
input_query = "what you remember about An Unlikely Heist: The Tale of Salvador Dali Art Theft"
output = chain.invoke(input_query)
print(output)


I remember that this article is about an unusual art theft involving a Salvador Dali drawing. A thief walked into a New York gallery, took the unprotected $150,000 drawing, and later returned it by mail. Greek fashion publicist Phivos Lampros Istavrioglou, 29, was identified as the thief through evidence such as his fingerprint on the mailing tube, relevant web searches from his Google account, and a photo of the drawing in the gallery that he posted on his Facebook page. He pleaded not guilty after being arrested and lured back to
